In [ ]:
import os
from glob import glob
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import datasets, transforms, models
from torchvision.models import resnet50

from sklearn.metrics import confusion_matrix
from tqdm import tqdm


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')
print(torch.cuda.get_device_name(0))

# Costum Image Dataset

In [ ]:
# Custom Dataset sınıfı
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, class_to_idx=None, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        self.class_to_idx = class_to_idx  # Burada class_to_idx'yi alıyoruz

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


# Klasörlerden resimleri ve etiketleri almak için fonksiyon
def get_image_paths_and_labels(root_dir):
    image_paths = []
    labels = []
    class_names = sorted(os.listdir(root_dir))  # alt klasör isimleri -> sınıf
    class_to_idx = {class_name: idx for idx, class_name in enumerate(class_names)}  # Sınıf -> indeks

    for class_name in class_names:
        class_path = os.path.join(root_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        for img_path in glob(os.path.join(class_path, '*')):
            image_paths.append(img_path)
            labels.append(class_to_idx[class_name])  # Sınıfın indeksini etiket olarak ekle
        
    return image_paths, labels, class_to_idx  # class_to_idx'yi döndür


# Transformlar
transformsForTrain = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transformsForVal = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

transformsForTest = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


# Klasör yolları
train_dir = 'Your_train_directory_path_here'  # Eğitim seti klasör yolu
val_dir = 'Your_val_directory_path_here'      # Doğrulama seti klasör yolu
test_dir = 'Your_test_directory_path_here'    # Test seti klasör yolu

# Eğitim seti
train_image_paths, train_labels, train_class_to_idx = get_image_paths_and_labels(train_dir)
train_data = CustomImageDataset(train_image_paths, train_labels, class_to_idx=train_class_to_idx, transform=transformsForTrain)

# Doğrulama seti
val_image_paths, val_labels, val_class_to_idx = get_image_paths_and_labels(val_dir)
val_data = CustomImageDataset(val_image_paths, val_labels, class_to_idx=val_class_to_idx, transform=transformsForVal)

# Test seti
test_image_paths, test_labels, test_class_to_idx = get_image_paths_and_labels(test_dir)
test_data = CustomImageDataset(test_image_paths, test_labels, class_to_idx=test_class_to_idx, transform=transformsForTest)


# DataLoader oluşturma
bs = 32
train_loader = DataLoader(train_data, batch_size=bs, shuffle=True)
val_loader = DataLoader(val_data, batch_size=bs, shuffle=False)
test_loader = DataLoader(test_data, batch_size=bs, shuffle=False)

# Eğitim, doğrulama ve test verisinin toplam uzunluğunu yazdır
print("Total images in train dataset:", len(train_data))
print("Total images in validation dataset:", len(val_data))
print("Total images in test dataset:", len(test_data))

# class_to_idx'yi kontrol etme
print("class_to_idx for train dataset:", train_class_to_idx)


- Sınıf Adları

In [ ]:
class_names = list(test_loader.dataset.class_to_idx.keys())
print(class_names)

- Sınıf Sayısı

In [ ]:
num_classes = len(set(train_labels))
print(num_classes)

# Model Oluşumu

In [ ]:
def Resnet50Model(num_classes):
    model = models.resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    print("Model created successfully")
    return model.to(device)

num_classes = len(train_class_to_idx)
model = Resnet50Model(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train Fonksiyonu

In [ ]:
def train_model(model, criterion, optimizer, train_loader, val_loader, device, num_epochs=10):
    best_loss = float('inf')
    
    for epoch in range(num_epochs):
        print(f"\n{'=' * 40}")
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"{'=' * 40}")
        
        # Eğitim
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(train_loader, desc="Training", leave=False):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        
        print(f"[Train] Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.4f}")

        # Doğrulama
        model.eval()
        val_loss = 0.0
        val_corrects = 0

        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc="Validation", leave=False):
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_corrects.double() / len(val_loader.dataset)

        print(f"[Val]   Loss: {val_loss:.4f} | Accuracy: {val_acc:.4f}")

        # En iyi modeli kaydetme
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), 'FaceDetection_best.pth')
            print(f"✅ Best model saved (Val Loss: {val_loss:.4f})")

    # Son modeli kaydet
    torch.save(model.state_dict(), 'FaceDetection_last.pth')
    print("\n🎉 Training completed. Final model saved as 'FaceDetection_last.pth'")
    return model

# Train Aşaması

In [ ]:
train_model(model, criterion, optimizer, train_loader, val_loader, device, num_epochs=50)

# Test Aşaması

In [ ]:
import torch
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Resnet50Model(num_classes)
model.load_state_dict(torch.load('FaceDetection_best.pth'))
model = model.to(device)
model.eval()

# Test işlemi için değişkenleri başlat
test_running_loss, test_running_corrects = 0.0, 0
all_preds = []
all_labels = []

# Test işlemi (Evaluation)
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        test_running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        test_running_corrects += torch.sum(preds == labels)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_loss = test_running_loss / len(test_loader.dataset)
test_acc = test_running_corrects.double() / len(test_loader.dataset)

print(f"✅ Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

cm = confusion_matrix(all_labels, all_preds)
class_names = list(test_loader.dataset.class_to_idx.keys())

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Tahmin Edilen Etiket")
plt.ylabel("Gerçek Etiket")
plt.title(f"Confusion Matrix\nTest Accuracy: {test_acc:.4f}")
plt.tight_layout()
plt.show()


# Rastgele 10 test görüntüsü ve tahminleri görselleştir

In [ ]:
import random

# 10 rastgele örnek seç
random_indices = random.sample(range(len(test_data)), 10)
images, labels = zip(*[test_data[i] for i in random_indices])

# Batch'i tek bir tensor olarak birleştir
images = torch.stack(images)  # Tensor'u birleştir

# Modeli tahmin yapmak için kullan
model.eval()
with torch.no_grad():
    outputs = model(images.to(device))
    _, preds = torch.max(outputs, 1)

# Görselleri ve tahminleri göster
plt.figure(figsize=(12, 10))

# Görselleri sırayla yerleştir
for i in range(10):
    plt.subplot(2, 5, i + 1)  # 2 satır, 5 sütunlu bir ızgara
    image = images[i].permute(1, 2, 0).cpu().numpy()  # Tensor'u numpy array'e çevir
    image = np.clip(image, 0, 1)  # Görüntüyü normalleştir
    plt.imshow(image)
    
    # Gerçek etiket ve tahmin edilen etiket
    true_label = labels[i]
    predicted_label = preds[i]
    
    # Başlık kısmını düzenle
    plt.title(f"True: {class_names[true_label]}\nPred: {class_names[predicted_label]}", fontsize=10)
    plt.axis('off')

# Görsellerin düzgün şekilde hizalanması ve boşlukların artırılması
plt.subplots_adjust(wspace=0.4, hspace=0.4)
plt.tight_layout()
plt.show()


# Yüz tanıma için OpenCV kullanarak yüz tespiti


In [ ]:
import cv2
import matplotlib.pyplot as plt

# Görüntüyü yükle
img = cv2.imread('X.jpg') # Herhangi bir görüntü dosyasını yükleyebilirsiniz

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# OpenCV'nin hazır yüz dedektörünü yükle
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Yüzleri tespit et
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

# Yüzlerin etrafına kutu çiz
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Sonucu göster
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

### Model ile yüz tespiti birlikte, yüzün etrafına bir dikdörtgen çizeceğiz.
### Yüz tespiti için ise eğittiğimiz modelin ağırlıklarını kullanacağız.

In [ ]:
from facenet_pytorch import MTCNN
from PIL import Image
import torch
from torchvision import transforms
import numpy as np
import cv2
import matplotlib.pyplot as plt


# Cihaz seçimi
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Modeli oluştur ve yükle
model = Resnet50Model(num_classes)
model.load_state_dict(torch.load('FaceDetection_best.pth', map_location=device))
model.to(device)
model.eval()

# Sınıf isimleri
class_names = ['Female', 'Male']

# MTCNN ile yüz tespiti
mtcnn = MTCNN(keep_all=True, device=device)

# Görüntüyü yükle (PIL formatı)
img = Image.open('X.jpg') # Herhangi bir resim dosyasını buraya koyabilirsiniz
plt.imshow(img)


# Yüzleri tespit et
boxes, _ = mtcnn.detect(img)

# Görüntüyü numpy olarak kopyala (cv2 için)
#img_np = np.array(img)
img_np = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
img_copy = img_np.copy()

transform = transforms.Compose([
    transforms.ToTensor(),
])

# Her yüz için sınıflandırma
if boxes is not None:
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)
        face = img.crop((x1, y1, x2, y2))
        face_tensor = transform(face).unsqueeze(0).to(device)

        with torch.no_grad():
            outputs = model(face_tensor)
            _, predicted = torch.max(outputs, 1)
            label = class_names[predicted.item()]

        # Kutuyu çiz ve etiketle
        cv2.rectangle(img_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img_copy, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

# Sonuç göster
plt.imshow(cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()
